In [1]:
import torch
! pip install datasets
from datasets import load_dataset
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm.notebook import tqdm
import random
!pip install gdown  # if not already installed
import gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.1 MB/s eta 0:00:00


# 1. Loading Model

In [2]:
class MoLFormerWithRegressionHead(nn.Module):
    def __init__(self, base_model):
        super(MoLFormerWithRegressionHead, self).__init__()
        self.base_model = base_model
        self.regression_head = nn.Linear(base_model.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        return self.regression_head(pooled_output)
# loading the fine-tuned MLM model
MODEL_URL = "https://drive.google.com/drive/folders/155k0aND9BfMUZT5tCHjvfPYkY7IuFMRm"
output = './unsupervised_model.pth'
gdown.download_folder(MODEL_URL, output=output, quiet=False)

gdown.download(MODEL_URL, output, quiet=False)
base_model = AutoModel.from_pretrained(output)
regression_model = MoLFormerWithRegressionHead(base_model)

Retrieving folder contents


Processing file 15CIJ2uwO3u1FlzWzVwrfHypE0xDPr2Kk config.json
Processing file 15BqT0pUcvke9yiE1-KslSMOf-BVQe-et model.safetensors


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=15CIJ2uwO3u1FlzWzVwrfHypE0xDPr2Kk
To: /content/unsupervised_model.pth/config.json
100%|██████████| 1.18k/1.18k [00:00<00:00, 1.05MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=15BqT0pUcvke9yiE1-KslSMOf-BVQe-et
From (redirected): https://drive.google.com/uc?id=15BqT0pUcvke9yiE1-KslSMOf-BVQe-et&confirm=t&uuid=0ce09215-e509-4598-b43b-e9697b1b71c6
To: /content/unsupervised_model.pth/model.safetensors
100%|██████████| 187M/187M [00:02<00:00, 67.5MB/s]
Download completed
/usr/local/lib/python3.11/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/drive/folders/155k0aND9BfMUZT5tC

The repository for ./unsupervised_model.pth contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/./unsupervised_model.pth.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


configuration_molformer.py:   0%|          | 0.00/7.60k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ibm/MoLFormer-XL-both-10pct:
- configuration_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


The repository for ./unsupervised_model.pth contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/./unsupervised_model.pth.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


modeling_molformer.py:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ibm/MoLFormer-XL-both-10pct:
- modeling_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [3]:
url = 'https://drive.google.com/uc?id=15Flh6v2fHlyz4ruJ8pi-wMHuAFV81OB8'
output = 'regression_model.pth'
gdown.download(url, output, quiet=False)
regression_model = torch.load(output)


Downloading...
From (original): https://drive.google.com/uc?id=15Flh6v2fHlyz4ruJ8pi-wMHuAFV81OB8
From (redirected): https://drive.google.com/uc?id=15Flh6v2fHlyz4ruJ8pi-wMHuAFV81OB8&confirm=t&uuid=007ab94c-a07d-4b31-9922-85c669412c15
To: /content/regression_model.pth
100%|██████████| 179M/179M [00:03<00:00, 53.7MB/s]
<ipython-input-3-af998adda667>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.ad

# 2. Calculate Influence Score

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd


class SMILESDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=256):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.tokenizer(item["SMILES"], padding="max_length", truncation=True, return_tensors="pt", max_length=self.max_length)
        encoding = {key: val.squeeze(0) for key, val in encoding.items()}
        encoding["labels"] = torch.tensor(item["label"])
        return encoding


def compute_gradient(model, loss_fn, inputs, labels, create_graph=False):

    model.zero_grad()
    preds = model(inputs)
    loss = loss_fn(preds, labels)
    loss.backward(create_graph=create_graph)
    grads = [p.grad.clone() for p in model.parameters() if p.requires_grad]
    return grads

def hvp(model, loss_fn, train_loader, vector, device='cuda'):
    """
    We'll do a mini-batch approximation:
      1) sample a batch from 'train_loader'
      2) compute gradient
      3) do a second backward pass with dot-product of gradient and v
    """
    # 1) pick one batch to approximate
    batch = next(iter(train_loader))
    inputs = batch["input_ids"].to(device)
    labels = batch["labels"].to(device)
    inputs, labels = inputs.to(device), labels.to(device)

    model.zero_grad()
    preds = model(inputs)
    loss = loss_fn(preds, labels)
    grads = torch.autograd.grad(loss, model.parameters(), create_graph=True)

    flat_grad = torch.cat([t.view(-1) for t in grads], dim=0)
    flat_vec = torch.cat([t.view(-1) for t in vector], dim=0)
    dot_val = (flat_grad * flat_vec).sum()
    #second backward pass to get Hessian-vector product
    hvp_grad = torch.autograd.grad(dot_val, model.parameters(), retain_graph=False)

    return hvp_grad



def lissa_iHVP(model, loss_fn, train_loader, test_grad, scale=1.0, recursion_depth=10, device='cuda'):
    """
    LiSSA approximation for iHVP = H^-1 * test_grad.

      x_0 = 0
      x_{t+1} = test_grad + (I - H)*x_t

    But we get H*x_t via hvp(...) each time.
    """

    model.eval()
    cur_estimate = [torch.zeros_like(p) for p in test_grad]  # x_0 = 0

    for _ in range(recursion_depth):
        hvp_est = hvp(model, loss_fn, train_loader, cur_estimate, device=device)

        # update: x_{t+1} = test_grad + (cur_estimate - hvp_est)*scale
        new_estimate = []
        for ce, hvp_e, tg in zip(cur_estimate, hvp_est, test_grad):
            tmp = ce - hvp_e
            tmp = scale * tmp
            tmp = tg + tmp
            new_estimate.append(tmp)
        cur_estimate = new_estimate

    return cur_estimate

def compute_influence_scores(
    model,
    loss_fn,
    train_loader,
    test_loader,
    external_loader,
    recursion_depth=10,
    scale=1.0,
    device='cuda'
):
    model.to(device)

    # Compute test_grad by summing/averaging over entire test set
    test_grad = approximate_gradient_on_loader(model, loss_fn, test_loader, device=device)

    # Approximate iHVP = H^-1 * test_grad
    iHVP = lissa_iHVP(
        model=model,
        loss_fn=loss_fn,
        train_loader=train_loader,
        test_grad=test_grad,
        recursion_depth=recursion_depth,
        scale=scale,
        device=device
    )

    influence_scores = []
    for idx, (batch) in enumerate(external_loader):
        ext_x = batch["input_ids"].to(device)
        ext_y = batch["labels"].to(device)
        ext_x, ext_y = ext_x.to(device), ext_y.to(device)
        ext_grad = compute_gradient(model, loss_fn, ext_x, ext_y, create_graph=False)

        iHVP = torch.cat([t.view(-1) for t in iHVP], dim=0)
        ext_grad = torch.cat([t.view(-1) for t in ext_grad], dim=0)
        dot_val = torch.sum(iHVP * ext_grad)

        influence = -dot_val.item()
        influence_scores.append((idx, influence))

    return influence_scores

def approximate_gradient_on_loader(model, loss_fn, data_loader, device='cuda'):
    """
    Compute the gradient of L wrt the model parameters,
    where L is the sum (or mean) over the entire data_loader.
    """
    model.eval()
    total_grad = None
    count = 0
    for batch in data_loader:
        x = batch["input_ids"].to(device)
        y = batch["labels"].to(device)
        grads = compute_gradient(model, loss_fn, x, y, create_graph=False)
        if total_grad is None:
            total_grad = grads
        else:
            for i in range(len(grads)):
                total_grad[i] += grads[i]
        count += 1

    for i in range(len(total_grad)):
        total_grad[i] /= float(count)
    return total_grad


def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = regression_model
    model.to(device)
    loss_fn = nn.MSELoss()


    DATASET_PATH = "scikit-fingerprints/MoleculeNet_Lipophilicity"
    MODEL_NAME = "ibm/MoLFormer-XL-both-10pct"
    BATCH_SIZE = 16
    dataset = load_dataset(DATASET_PATH)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    dataset_train, dataset_test = train_test_split(list(dataset["train"]), test_size=0.2, random_state=42, shuffle=True)
    train_dataset = SMILESDataset(dataset_train, tokenizer=tokenizer)
    test_dataset = SMILESDataset(dataset_test, tokenizer=tokenizer)

    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


    external_dataset = pd.read_csv("./External-Dataset_for_Task2.csv")
    external_dataset = external_dataset.to_dict(orient="records")
    external_dataset = SMILESDataset(external_dataset, tokenizer=tokenizer)
    external_dataloader = DataLoader(external_dataset, batch_size=1, shuffle=False)

    influence_scores = compute_influence_scores(
        model=model,
        loss_fn=loss_fn,
        train_loader=train_dataloader,
        test_loader=test_dataloader,
        external_loader=external_dataloader,
        recursion_depth=10,
        scale=1.0,
        device=device
    )

    # Sort and find top-k
    influence_scores.sort(key=lambda x: x[1], reverse=True)
    top_k = 100
    top_k_indices = [x[0] for x in influence_scores[:top_k]]
    return top_k_indices

top_k = main()


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.S

# 3. Run Model on Extended Dataset

In [6]:
DATASET_PATH = "scikit-fingerprints/MoleculeNet_Lipophilicity"
MODEL_NAME = "ibm/MoLFormer-XL-both-10pct"
BATCH_SIZE = 16
dataset = load_dataset(DATASET_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
dataset_train, dataset_test = train_test_split(list(dataset["train"]), test_size=0.2, random_state=42, shuffle=True)
train_dataset = SMILESDataset(dataset_train, tokenizer=tokenizer)
test_dataset = SMILESDataset(dataset_test, tokenizer=tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


external_dataset = pd.read_csv("./External-Dataset_for_Task2.csv")
external_dataset = external_dataset.to_dict(orient="records")
external_dataset = SMILESDataset(external_dataset, tokenizer=tokenizer)
external_dataloader = DataLoader(external_dataset, batch_size=1, shuffle=False)

top_k_samples = torch.utils.data.Subset(external_dataset, top_k)

combined_dataset = torch.utils.data.ConcatDataset([train_dataset, top_k_samples])
print(len(combined_dataset))
print(len(train_dataset))
combined_loader = DataLoader(combined_dataset, batch_size=BATCH_SIZE, shuffle=True)

EPOCHS = 10
LR = 5e-5

optimizer = torch.optim.AdamW(regression_model.parameters(), lr=LR)
loss_fn = nn.MSELoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for epoch in range(EPOCHS):
    regression_model.train()
    total_loss = 0
    for batch in tqdm(combined_loader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = regression_model(input_ids, attention_mask=attention_mask).squeeze()
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_dataloader)}")



3460
3360


  0%|          | 0/217 [00:00<?, ?it/s]

Epoch 1, Loss: 0.20619947724044324


  0%|          | 0/217 [00:00<?, ?it/s]

Epoch 2, Loss: 0.1608981761726595


...

Epoch 8, Loss: 0.09300866896907488


  0%|          | 0/217 [00:00<?, ?it/s]

Epoch 9, Loss: 0.08812659127371651


  0%|          | 0/217 [00:00<?, ?it/s]

Epoch 10, Loss: 0.07888558286109142


# 4. Test

In [7]:
from sklearn.metrics import mean_squared_error

regression_model.eval()
predictions, actuals = [], []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = regression_model(input_ids, attention_mask=attention_mask).squeeze()
        predictions.extend(outputs.cpu().numpy())
        actuals.extend(labels.cpu().numpy())

mse = mean_squared_error(actuals, predictions)
print("MSE:", mse)

MSE: 0.41953555225390887
